Import libs

In [54]:
from neo4j import GraphDatabase
import pathlib

Create connection

In [ ]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
                
        return response
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="vovse")

Delete everything in graph

In [74]:
query = '''MATCH (n) DETACH DELETE n'''
out = conn.query(query)

Load computer data

In [76]:
query = '''
CALL apoc.load.json('%s/../jsondata/0001-computers.json') YIELD value
UNWIND value.computers AS c
MERGE (computer:Computer {sid:c.SID})
SET computer += c.Properties
''' % pathlib.Path().absolute().as_uri()
out = conn.query(query)

In [77]:
query = '''MATCH (n) RETURN n'''
out = conn.query(query)
out

[<Record n=<Node id=6 labels=frozenset({'Computer'}) properties={'PrimaryGroupSid': 'S-1-5-21-3712873460-3785406585-1658148875-515', 'haslaps': False, 'operatingsystem': 'Windows Server 2019 Datacenter Evaluation', 'unconstraineddelegation': False, 'enabled': True, 'sid': 'S-1-5-21-3712873460-3785406585-1658148875-1111', 'tier': 1, 'domain': 'HOT.LOCAL', 'pwdlastset': 1592308245, 'name': 'HOT-JUMPHOST.HOT.LOCAL', 'distinguishedname': 'CN=HOT-JUMPHOST,OU=Jumphost,DC=hot,DC=local', 'serviceprincipalnames': ['WSMAN/hot-jumphost', 'WSMAN/hot-jumphost.hot.local', 'TERMSRV/HOT-JUMPHOST', 'TERMSRV/hot-jumphost.hot.local', 'RestrictedKrbHost/HOT-JUMPHOST', 'HOST/HOT-JUMPHOST', 'RestrictedKrbHost/hot-jumphost.hot.local', 'HOST/hot-jumphost.hot.local'], 'lastlogontimestamp': 1592307353}>>,
 <Record n=<Node id=7 labels=frozenset({'Computer'}) properties={'PrimaryGroupSid': 'S-1-5-21-3712873460-3785406585-1658148875-515', 'haslaps': False, 'operatingsystem': 'Windows Server 2019 Datacenter Evaluat